In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

# Loading Our Data

In [ ]:
data = pd.read_csv('/kaggle/input/vehicle-dataset-from-cardekho/car data.csv')
data.head()

In [ ]:
data.shape #Checking number of rows and cols

In [ ]:
data.columns

In [ ]:
data.describe().T # Gives you overview of the data

In [ ]:
data.info() 

## Null checking

In [ ]:
data.isna().sum()

No null values, so far so good..

# Separating numerical and categorical collumns

In [ ]:
all_cols = data.columns
numerical_cols = data._get_numeric_data().columns.to_list()
categorical_cols = list(set(all_cols)- set(numerical_cols))

In [ ]:
numerical_cols, categorical_cols

In [ ]:
categorical_cols

In [ ]:
[data[x].unique() for x in categorical_cols[1:] ] #excluding car name which is first element

fuel_type has three unique values: Petrol / Diesel / CNG  
Seller type : Dealer / Individual  
Transmission: Manual / Automatic

In [ ]:
# we don't need car name so i am gonna drop it right away
data.drop(['Car_Name'], inplace=True, axis=1)

In [ ]:
data.head()

Now, for the year column, what we need is the difference between current year (2020) and the year when car was manufactured. This will give us the number of years how old car is.

In [ ]:
# car's age
data['Car_Age'] = 2020 - data.Year
data.head()

In [ ]:
# Dropping year column, we dont need it now
data.drop(['Year'], inplace=True, axis=1)

# One Hot Encoding categorical columns

In [ ]:
data = pd.get_dummies(data, drop_first=True)# Get dummies will return OHE columns
data.head()

In [ ]:
data.shape

# Finding Correlation

In [ ]:
data.corr()

In [ ]:
import seaborn as sns
plt.figure(figsize=(18,8))
sns.heatmap(data.corr(),annot=True);

# Getting our data into X and y

In [ ]:
y = data.pop('Selling_Price') #will pop the sel price collumn and drop it in y
X = data #remaining data

In [ ]:
X.head()

In [ ]:
y.head()

# Checking Feature importance

which features are important which are not

In [ ]:
# Feature importance
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(X, y)

In [ ]:
plt.figure(figsize=(18,8))
sns.barplot(x=data.columns, y=model.feature_importances_);

so as you can see, present price is the most important feature then comes diesel based cars so on and so fourth

# Train-Test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape

# Model Building: Linear Regression

In [ ]:
#Create the regressor: reg
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

In [ ]:
#Fit the regressor to the training data
reg.fit(X_train, y_train)

In [ ]:
# Predict on the test data: y_pred
y_pred = reg.predict(X_test)

In [ ]:
# Compute and print RMSE
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))

# Cross validation with linear regression

In [ ]:
from sklearn.model_selection import cross_val_score

# Define the regression_model_cv function, which takes a fitted model as one parameter. The k = 5 hyperparameter gives the number of folds.
def regression_model_cv(model, k=5):
    scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=k)
    rmse = np.sqrt(-scores)
    print('Reg rmse:', rmse)
    print('Reg mean:', rmse.mean ())

In [ ]:
regression_model_cv(LinearRegression())


In [ ]:
#Use the regression_model_cv function on the LinearRegression() model with 3 folds and then 6 folds, as shown in the following code snippet, for 3 folds:
regression_model_cv(LinearRegression(), k=3)

In [ ]:
# Now, test the values for 6 folds
regression_model_cv(LinearRegression(), k=6)

So with k=5 we get best results

# Regularization: Ridge and Lasso

In [ ]:
#We begin by setting Ridge() as a parameter for regression_model_cv
from sklearn.linear_model import Ridge
regression_model_cv(Ridge())

In [ ]:
# Now, set Lasso() as the parameter for regression_model_cv:
from sklearn.linear_model import Lasso
regression_model_cv(Lasso())

With ridge, we get slightly better result,great!

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor()

In [ ]:
# Hyperparameters
n_estimators = [100,200,300,500]
max_features = ['auto', 'sqrt']
max_depth = [5, 10, 15, 20]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
grid = {'n_estimators': n_estimators,
        'max_features': max_features,
        'max_depth': max_depth
}

In [ ]:
clf_cv = RandomizedSearchCV(estimator=clf, param_distributions=grid, scoring='neg_mean_squared_error', n_iter=10, cv=5, verbose=2, random_state=42)

In [ ]:
clf_cv.fit(X_train, y_train)

In [ ]:
preds = clf_cv.predict(X_test)
preds

In [ ]:
from sklearn.metrics import mean_squared_error
rmse_value = mean_squared_error(y_test, preds, squared=False)
rmse_value

In [ ]:
sns.distplot(y_test-preds);

In [ ]:
plt.scatter(y_test, preds);